In [50]:
# import csv

# # Path to your input text file
# input_file = 'val.txt'

# # Path to your output CSV file
# output_file = 'val.csv'

# # Delimiter used in your text file (e.g., ',', '\t', ';', etc.)
# delimiter = ','  # Change this to the delimiter used in your file

# # Open the input text file for reading with UTF-8 encoding
# with open(input_file, 'r', encoding='utf-8') as txt_file:
#     # Read the lines from the text file
#     lines = txt_file.readlines()

# # Open the output CSV file for writing with UTF-8 encoding
# with open(output_file, 'w', newline='', encoding='utf-8') as csv_file:
#     # Create a CSV writer object
#     csv_writer = csv.writer(csv_file)
    
#     # Iterate over the lines from the text file
#     for line in lines:
#         # Split the line into fields based on the delimiter
#         fields = line.strip().split(delimiter)
        
#         # Write the fields to the CSV file
#         csv_writer.writerow(fields)

# print("Conversion completed successfully.")


In [51]:
import pandas as pd

In [52]:
import pandas as pd
import re

# Path to the file
file_path = 'val_raw.csv'

# Read the entire file into a single string
with open(file_path, 'r', encoding='utf-8') as file:
    data = file.read()

# Split the data into entries (empty lines or specific separators might be used to split)
entries = data.split('""\n')

# Prepare lists to hold columns
texts = []
tags = []

# Process each entry
for entry in entries:
    if entry.strip():  # Skip any empty entries
        # Find all tag patterns and extract them
        tag_matches = re.findall(r'\{([^}]+)\}', entry)
        # Remove the tags from the text to isolate the review text
        text = re.sub(r'\{[^}]+\}', '', entry).strip()

        # Append the cleaned text and tags to the respective lists
        texts.append(text)
        tags.append(tag_matches)

# Create a DataFrame
df_train = pd.DataFrame({
    'Text': texts,
    'Tags': tags
})

# Display the DataFrame
df_train.head()


,Text,Tags
0,"﻿#1\n- 60k/1con- Tu hài to, siêu béo siêu ngon...","[FOOD#PRICES, positive, FOOD#QUALITY, positive]"
1,#2\n- Ăn hai người no căng :3- Nhiều đồ ăn kèm...,"[FOOD#STYLE&OPTIONS, positive, RESTAURANT#GENE..."
2,#3\n- Ấn tượng đầu là quán siêu to siêu thoáng...,"[AMBIENCE#GENERAL, positive, FOOD#STYLE&OPTION..."
3,#4\n- Bánh ăn thơm và vừa miệng. Gìon rụ...,"[FOOD#QUALITY, positive, RESTAURANT#GENERAL, p..."
4,"#5\n- Bánh siêu ngon, siêu mềm.Tự tay anh c...","[FOOD#QUALITY, positive]"


In [53]:
import pandas as pd

# Define the mapping of sentiment values to numeric labels
sentiment_mapping = {'None': 0, 'negative': 1, 'neutral': 2, 'positive': 3}

# Assuming df_train contains your DataFrame with 'Text' and 'Tags' columns

# Extract unique aspects
aspects = set()
for tags_list in df_train['Tags']:
    for tag in tags_list:
        aspect, sentiment = tag.split(', ')
        aspects.add(aspect.strip('{}'))

# Create a dictionary to hold data for new DataFrame
data_dict = {'Text': df_train['Text']}
for aspect in aspects:
    data_dict[aspect] = [0] * len(df_train)  # Initialize with 0 instead of None

# Fill the sentiment values
for i, tags_list in enumerate(df_train['Tags']):
    for tag in tags_list:
        aspect, sentiment = tag.split(', ')
        aspect = aspect.strip('{}')
        data_dict[aspect][i] = sentiment_mapping[sentiment]  # Map sentiment to numeric label

# Create the new DataFrame
new_df_train = pd.DataFrame(data_dict)

# Display the new DataFrame
new_df_train.head()


,Text,FOOD#PRICES,DRINKS#STYLE&OPTIONS,FOOD#STYLE&OPTIONS,SERVICE#GENERAL,RESTAURANT#MISCELLANEOUS,DRINKS#QUALITY,RESTAURANT#PRICES,LOCATION#GENERAL,DRINKS#PRICES,RESTAURANT#GENERAL,AMBIENCE#GENERAL,FOOD#QUALITY
0,"﻿#1\n- 60k/1con- Tu hài to, siêu béo siêu ngon...",3,0,0,0,0,0,0,0,0,0,0,3
1,#2\n- Ăn hai người no căng :3- Nhiều đồ ăn kèm...,3,0,3,0,0,0,0,0,0,3,0,0
2,#3\n- Ấn tượng đầu là quán siêu to siêu thoáng...,3,0,3,3,0,0,0,0,0,0,3,3
3,#4\n- Bánh ăn thơm và vừa miệng. Gìon rụ...,0,0,0,0,0,0,0,0,0,3,0,3
4,"#5\n- Bánh siêu ngon, siêu mềm.Tự tay anh c...",0,0,0,0,0,0,0,0,0,0,0,3


In [54]:
new_df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1290 entries, 0 to 1289
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Text                      1290 non-null   object
 1   FOOD#PRICES               1290 non-null   int64 
 2   DRINKS#STYLE&OPTIONS      1290 non-null   int64 
 3   FOOD#STYLE&OPTIONS        1290 non-null   int64 
 4   SERVICE#GENERAL           1290 non-null   int64 
 5   RESTAURANT#MISCELLANEOUS  1290 non-null   int64 
 6   DRINKS#QUALITY            1290 non-null   int64 
 7   RESTAURANT#PRICES         1290 non-null   int64 
 8   LOCATION#GENERAL          1290 non-null   int64 
 9   DRINKS#PRICES             1290 non-null   int64 
 10  RESTAURANT#GENERAL        1290 non-null   int64 
 11  AMBIENCE#GENERAL          1290 non-null   int64 
 12  FOOD#QUALITY              1290 non-null   int64 
dtypes: int64(12), object(1)
memory usage: 131.1+ KB


In [55]:
new_df_train['Text'] = new_df_train['Text'].str.replace(r'#[0-9]+\n', '').str.replace('\n', '')
new_df_train.to_csv("val.csv", index= False)


C:\Users\FPTSHOP\AppData\Local\Temp\ipykernel_21120\1733955230.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  new_df_train['Text'] = new_df_train['Text'].str.replace(r'#[0-9]+\n', '').str.replace('\n', '')


In [56]:
df = pd.read_csv("val.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1290 entries, 0 to 1289
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Text                      1290 non-null   object
 1   FOOD#PRICES               1290 non-null   int64 
 2   DRINKS#STYLE&OPTIONS      1290 non-null   int64 
 3   FOOD#STYLE&OPTIONS        1290 non-null   int64 
 4   SERVICE#GENERAL           1290 non-null   int64 
 5   RESTAURANT#MISCELLANEOUS  1290 non-null   int64 
 6   DRINKS#QUALITY            1290 non-null   int64 
 7   RESTAURANT#PRICES         1290 non-null   int64 
 8   LOCATION#GENERAL          1290 non-null   int64 
 9   DRINKS#PRICES             1290 non-null   int64 
 10  RESTAURANT#GENERAL        1290 non-null   int64 
 11  AMBIENCE#GENERAL          1290 non-null   int64 
 12  FOOD#QUALITY              1290 non-null   int64 
dtypes: int64(12), object(1)
memory usage: 131.1+ KB
